# 📊 Automating the Collection of EXPLAIN Plans and Runtime Metrics in Db2 LUW
This notebook automates the process of running a SQL query in Db2 and capturing its execution details using EXPLAIN tables and activity event monitors. The workflow includes:

1. **Setup of Activity Event Monitor:** Configures an event monitor in Db2 to collect runtime metrics such as execution time and resource usage during query execution.
2. **Creation of EXPLAIN Tables:** Sets up EXPLAIN tables in Db2 to store detailed execution plans for SQL queries.
3. **Query Execution:** Runs a SQL query to gather execution metrics and analyze its performance.
4. **EXPLAIN Plan Generation:** Captures the execution plan of the query to understand how Db2 will process the operation.
5. **Exporting Data:** Exports the collected EXPLAIN and activity monitor data as CSV files for external analysis or reporting.

For detailed setup instructions and guidance on running this notebook, refer to the [README.md](./README.md) file in the same directory.

This notebook provides a fully automated approach for analyzing query performance and gathering runtime metrics, helping database administrators and developers optimize their queries effectively.


In [123]:
import os
from dotenv import dotenv_values
import json
import shutil
import pandas as pd

In [124]:
# Set display options to show the full content in the 'sql' column
pd.set_option('display.max_colwidth', None)  # Show full column width without truncation
pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame from wrapping across lines

# 🔍 Enter the Filename of a List of SQL Queries - each query needs to be in a single line

In [125]:
# Define the input SQL file
input_file = "queries.sql"  # Replace with your actual file name

# Read the SQL queries from the file, skipping empty lines and removing ending semicolons
with open(input_file, "r") as file:
    sql_queries = [
        line.strip().rstrip(';')  # Strip whitespace and remove trailing semicolons
        for line in file
        if line.strip()  # Skip empty lines
    ]

# Construct the DataFrame
df_queries = pd.DataFrame({
    "queryid": range(1, len(sql_queries) + 1),
    "sql": sql_queries
})

In [126]:
df_queries

,queryid,sql
0,1,"SELECT TPCDS.STORE_SALES.SS_WHOLESALE_COST , TPCDS.STORE_RETURNS.SR_NET_LOSS FROM TPCDS.STORE_SALES INNER JOIN TPCDS.STORE_RETURNS ON TPCDS.STORE_RETURNS.SR_TICKET_NUMBER = TPCDS.STORE_SALES.SS_TICKET_NUMBER AND TPCDS.STORE_RETURNS.SR_ITEM_SK = TPCDS.STORE_SALES.SS_ITEM_SK WHERE TPCDS.STORE_SALES.SS_STORE_SK = 2 AND TPCDS.STORE_RETURNS.SR_RETURNED_DATE_SK = 2451680"
1,2,"SELECT TPCDS.CATALOG_RETURNS.CR_RETURNING_ADDR_SK , TPCDS.CATALOG_SALES.CS_SALES_PRICE FROM TPCDS.CATALOG_RETURNS INNER JOIN TPCDS.CATALOG_SALES ON TPCDS.CATALOG_SALES.CS_ORDER_NUMBER = TPCDS.CATALOG_RETURNS.CR_ORDER_NUMBER AND TPCDS.CATALOG_SALES.CS_ITEM_SK = TPCDS.CATALOG_RETURNS.CR_ITEM_SK WHERE TPCDS.CATALOG_SALES.CS_NET_PAID <= +00456.68 AND TPCDS.CATALOG_RETURNS.CR_REFUNDED_HDEMO_SK <= 939"
2,3,"SELECT TPCDS.STORE_SALES.SS_EXT_LIST_PRICE , TPCDS.STORE_RETURNS.SR_STORE_CREDIT FROM TPCDS.STORE_SALES INNER JOIN TPCDS.STORE_RETURNS ON TPCDS.STORE_RETURNS.SR_CUSTOMER_SK = TPCDS.STORE_SALES.SS_CUSTOMER_SK AND TPCDS.STORE_RETURNS.SR_ITEM_SK = TPCDS.STORE_SALES.SS_ITEM_SK WHERE TPCDS.STORE_SALES.SS_ADDR_SK >= 8970 AND TPCDS.STORE_RETURNS.SR_CUSTOMER_SK <= 93014"
3,4,"SELECT SR_ADDR_SK, SR_CDEMO_SK, SR_RETURN_AMT, SR_RETURN_AMT_INC_TAX, SR_RETURN_TIME_SK, SR_REVERSED_CHARGE, SR_STORE_CREDIT, SS_EXT_DISCOUNT_AMT, SS_SOLD_DATE_SK, SS_LIST_PRICE, SS_QUANTITY, SS_SALES_PRICE, SS_SOLD_TIME_SK, SS_WHOLESALE_COST FROM TPCDS.STORE_RETURNS JOIN TPCDS.STORE_SALES ON SR_ITEM_SK = SS_ITEM_SK AND SR_CUSTOMER_SK = SS_CUSTOMER_SK AND SS_TICKET_NUMBER = SR_TICKET_NUMBER"


In [127]:
# Define the top-level output directory
top_level_dir = os.path.join(os.getcwd(), "output")

# If the top-level directory exists, delete it and recreate
if os.path.exists(top_level_dir):
    shutil.rmtree(top_level_dir)
os.makedirs(top_level_dir)

Loading Db2 Magic Commands Notebook Extension

In [128]:
# Enable Db2 Magic Commands Extensions for Jupyter Notebook
if not os.path.isfile('db2.ipynb'):
    os.system('wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb')
%run db2.ipynb

Db2 Extensions Loaded. Version: 2024-09-16


Connect to Db2

In [129]:
db2creds = dotenv_values('.env')
%sql CONNECT CREDENTIALS db2creds

Connection successful. tpcds @ localhost 


In [130]:
%sql CALL ADMIN_CMD("UPDATE DATABASE CONFIGURATION USING SECTION_ACTUALS BASE")

b. Deactivate event monitor, `ACTEVMON`

In [131]:
%sql SET EVENT MONITOR ACTEVMON STATE 0

Command completed.


b. Drop Existing Tables for event monitor `ACTEVMON`

In [132]:
%%sql -q
DROP TABLE ACTIVITYMETRICS_ACTEVMON;
DROP TABLE ACTIVITYSTMT_ACTEVMON;
DROP TABLE ACTIVITYVALS_ACTEVMON;
DROP TABLE ACTIVITY_ACTEVMON;
DROP TABLE CONTROL_ACTEVMON;

c. Install Explain Tables

In [133]:
%%sql
CALL SYSINSTALLOBJECTS('EXPLAIN', 'D', NULL, 'DB2INST1');
CALL SYSINSTALLOBJECTS('EXPLAIN', 'C', NULL, 'DB2INST1');

Command completed.


d. Alter Workload to Collect Activity Data


In [134]:
%sql ALTER WORKLOAD SYSDEFAULTUSERWORKLOAD COLLECT ACTIVITY DATA ON ALL WITH DETAILS, SECTION

Command completed.


e. Drop and re-create a New Event Monitor, `ACTEVMON`

In [135]:
%%sql 
DROP EVENT MONITOR ACTEVMON;
CREATE EVENT MONITOR ACTEVMON FOR ACTIVITIES WRITE TO TABLE;

Command completed.


In [136]:
# Activate event monitor
%sql SET EVENT MONITOR ACTEVMON STATE 1;

Command completed.


In [137]:
# Set client info
%sql CALL WLM_SET_CLIENT_INFO(NULL,NULL,NULL,'queryid1',NULL);

[None, None, None, 'queryid1', None]

In [138]:
%%capture
%%sql -q
SELECT STORE_SALES.SS_WHOLESALE_COST, STORE_RETURNS.SR_NET_LOSS
FROM STORE_SALES
INNER JOIN STORE_RETURNS
ON STORE_RETURNS.SR_TICKET_NUMBER = STORE_SALES.SS_TICKET_NUMBER
AND STORE_RETURNS.SR_ITEM_SK = STORE_SALES.SS_ITEM_SK
WHERE STORE_SALES.SS_STORE_SK = 2
AND STORE_RETURNS.SR_RETURNED_DATE_SK = 2451680

In [139]:
# Deactivate the event monitor to ensure its data is written to the activity tables
%sql SET EVENT MONITOR ACTEVMON STATE 0

Command completed.


In [140]:
result = %sql SELECT a.APPL_ID, a.UOW_ID, a.ACTIVITY_ID \
    FROM ACTIVITY_ACTEVMON a \
    WHERE a.ACTIVITY_TYPE = 'READ_DML' AND a.TPMON_ACC_STR = 'queryid1'

In [141]:
   
appl_id = result['APPL_ID'].iloc[0]
uow_id = result.at[0, 'UOW_ID'].item()
activity_id = result.at[0, 'ACTIVITY_ID'].item()
event_monitor = 'ACTEVMON'
schema = 'DB2INST1'

In [142]:

print(result)

                        APPL_ID  UOW_ID  ACTIVITY_ID
0  127.0.0.1.36406.250225023906      77            1


In [143]:
print('appl_id: ', appl_id)
print('uow_id: ', uow_id)
print('activity_id: ', activity_id)

appl_id:  127.0.0.1.36406.250225023906
uow_id:  77
activity_id:  1


In [144]:
%%capture explain_output
sql = f'''"CALL EXPLAIN_FROM_ACTIVITY('{appl_id}', '{uow_id}', '{activity_id}', '{event_monitor}', '{schema}', null, null, null, null, null)"'''
_ = ! db2 "connect to TPCDS" 

explain = %system db2 {sql}

In [145]:
# Initialize variables
explain_time = None
source_name = None
source_schema = None

# Iterate through the list
for i in range(len(explain)):
    if "EXPLAIN_TIME" in explain[i]:
        explain_time = explain[i + 1].split(":")[-1].strip()
    elif "SOURCE_NAME" in explain[i]:
        source_name = explain[i + 1].split(":")[-1].strip()
    elif "SOURCE_SCHEMA" in explain[i]:
        source_schema = explain[i + 1].split(":")[-1].strip()

# Print extracted values
print("EXPLAIN_TIME:", explain_time)
print("SOURCE_NAME:", source_name)
print("SOURCE_SCHEMA:", source_schema)

EXPLAIN_TIME: 2025-02-24-18.39.39.920008
SOURCE_NAME: SYSSH200
SOURCE_SCHEMA: NULLID


In [146]:
explain_time

'2025-02-24-18.39.39.920008'

In [147]:
from dotenv import dotenv_values

# Load environment variables from the .env file
db2creds = dotenv_values('.env')

# Extract the database name
database_name = db2creds.get("database")  # Use .get() to avoid KeyError if the key is missing

# Print the extracted database name
print("Database Name:", database_name)

Database Name: tpcds


# Generate Explain and Export the Explain output

In [148]:
# Define the subdirectory for a specific explain_time output
outputdir = os.path.join(top_level_dir, f"{explain_time}")
os.makedirs(outputdir, exist_ok=True)  # Create the subdirectory if it doesn't exist

# If the directory exists, delete its contents
for filename in os.listdir(outputdir):
    file_path = os.path.join(outputdir, filename)
    if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)  # Remove files and symlinks
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)  # Remove subdirectories

# Define output file
explain_output = "explain.out"

db2exfmt_cmd = f'''db2exfmt -d "{database_name}" -w "{explain_time}" -n "{source_name}" -s "{source_schema}" -# 0 -o "{outputdir}/{explain_output}"'''
print(db2exfmt_cmd)

# Uncomment the following line to execute the command (if running in a shell environment)
os.system(db2exfmt_cmd)

# Load the dictionary from the JSON file
with open("export_sql.json", "r") as json_file:
    export_sql_statements = json.load(json_file)

print("SQL statements have been loaded from sql_statements.json")

# Loop through each table in the dictionary
for table_name, query in export_sql_statements.items():
    print(f"Processing table: {table_name}")

    # Execute the dynamically generated SQL using %sql magic
    df_result = %sql {query}

    # Convert result to Pandas DataFrame
    # df_result = df_result.DataFrame()

    # Save to CSV without including the index in the "explain" directory
    output_file_path = os.path.join(outputdir, f"{table_name}.csv")
    df_result.to_csv(output_file_path, index=False)

    print(f"Saved {table_name} data to {output_file_path}")

db2exfmt -d "tpcds" -w "2025-02-24-18.39.39.920008" -n "SYSSH200" -s "NULLID" -# 0 -o "/home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-18.39.39.920008/explain.out"


DB2 Universal Database Version 11.5, 5622-044 (c) Copyright IBM Corp. 1991, 2019
Licensed Material - Program Property of IBM
IBM DATABASE 2 Explain Table Format Tool

Connect to Database Successful.
Output is in /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-18.39.39.920008/explain.out.
Executing Connect Reset -- Connect Reset was Successful.


Connecting to the Database.
SQL statements have been loaded from sql_statements.json
Processing table: ACTIVITYSTMT_ACTEVMON
Saved ACTIVITYSTMT_ACTEVMON data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-18.39.39.920008/ACTIVITYSTMT_ACTEVMON.csv
Processing table: ACTIVITY_ACTEVMON
Saved ACTIVITY_ACTEVMON data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-18.39.39.920008/ACTIVITY_ACTEVMON.csv
Processing table: EXPLAIN_ACTUALS
Saved EXPLAIN_ACTUALS data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-18.39.39.920008/EXPLAIN_ACTUALS.csv
Processing table: EXPLAIN_INSTANCE
Saved EXPLAIN_INSTANCE data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-18.39.39.920008/EXPLAIN_INSTANCE.csv
Processing table: EXPLAIN_OBJECT
Saved EXPLAIN_OBJECT data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-18.39.39.920008/EXPLAIN_OBJECT.csv
Processing table: EXPLAIN_OPERATOR
Saved EXPLAIN_OPERATOR data to /home

In [149]:
%sql CONNECT RESET

Connection closed.
